In [25]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, when, avg, concat, expr

In [2]:
# Create Spark session
spark = SparkSession.builder.appName("FilterAddColumnRename").getOrCreate()

23/08/31 08:40:16 WARN Utils: Your hostname, riyaz-Aspire-VX5-591G resolves to a loopback address: 127.0.1.1; using 192.168.1.164 instead (on interface enp3s0)
23/08/31 08:40:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/31 08:40:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/31 08:40:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
# Load the occupation dataset into a Spark DataFrame
occupation = spark.read.csv("./occupation.csv", header=True, inferSchema=True)

In [4]:
# Create a temporary view to use Spark SQL

occupation.createOrReplaceTempView("occupation_view")

### Additional questions:

Use both spark SQL and Pyspark to obtain answer wherever relevant

#### Filter out rows where the age is greater than 30 and create a new DataFrame. Then, add a new column named "is_elderly" with a value of "True" for these rows and "False" otherwise.Rename the "gender" column to "sex".

In [6]:
# Spark SQL

# Assuming you have the "occupation_view" SQL view
filtered_rows_sql = spark.sql("SELECT * FROM occupation_view WHERE age > 30")

# Add a new column "is_elderly" with a value of True or False
filtered_rows_sql = filtered_rows_sql.withColumn("is_elderly", when(col("age") > 60, lit(True)).otherwise(lit(False)))

# Rename the "gender" column to "sex"
filtered_rows_sql = filtered_rows_sql.withColumnRenamed("gender", "sex")

# Show the resulting DataFrame
filtered_rows_sql.show()

+-------+---+---+-------------+--------+----------+
|user_id|age|sex|   occupation|zip_code|is_elderly|
+-------+---+---+-------------+--------+----------+
|      2| 53|  F|        other|   94043|     false|
|      5| 33|  F|        other|   15213|     false|
|      6| 42|  M|    executive|   98101|     false|
|      7| 57|  M|administrator|   91344|     false|
|      8| 36|  M|administrator|   05201|     false|
|     10| 53|  M|       lawyer|   90703|     false|
|     11| 39|  F|        other|   30329|     false|
|     13| 47|  M|     educator|   29206|     false|
|     14| 45|  M|    scientist|   55106|     false|
|     15| 49|  F|     educator|   97301|     false|
|     18| 35|  F|        other|   37212|     false|
|     19| 40|  M|    librarian|   02138|     false|
|     20| 42|  F|    homemaker|   95660|     false|
|     25| 39|  M|     engineer|   55107|     false|
|     26| 49|  M|     engineer|   21044|     false|
|     27| 40|  F|    librarian|   30030|     false|
|     28| 32

In [7]:
# Pyspark

# Assuming you have the "occupation" DataFrame
filtered_occupation = occupation.filter(col("age") > 30)

# Add a new column "is_elderly" with a value of True or False
filtered_occupation = filtered_occupation.withColumn("is_elderly", when(col("age") > 30, lit(True)).otherwise(lit(False)))

# Rename the "gender" column to "sex"
filtered_occupation = filtered_occupation.withColumnRenamed("gender", "sex")

# Show the resulting DataFrame
filtered_occupation.show()

+-------+---+---+-------------+--------+----------+
|user_id|age|sex|   occupation|zip_code|is_elderly|
+-------+---+---+-------------+--------+----------+
|      2| 53|  F|        other|   94043|      true|
|      5| 33|  F|        other|   15213|      true|
|      6| 42|  M|    executive|   98101|      true|
|      7| 57|  M|administrator|   91344|      true|
|      8| 36|  M|administrator|   05201|      true|
|     10| 53|  M|       lawyer|   90703|      true|
|     11| 39|  F|        other|   30329|      true|
|     13| 47|  M|     educator|   29206|      true|
|     14| 45|  M|    scientist|   55106|      true|
|     15| 49|  F|     educator|   97301|      true|
|     18| 35|  F|        other|   37212|      true|
|     19| 40|  M|    librarian|   02138|      true|
|     20| 42|  F|    homemaker|   95660|      true|
|     25| 39|  M|     engineer|   55107|      true|
|     26| 49|  M|     engineer|   21044|      true|
|     27| 40|  F|    librarian|   30030|      true|
|     28| 32

#### Calculate the average age of male and female users separately. Present the result in a new DataFrame with columns "gender" and "avg_age".

In [8]:
# Spark SQL

spark.sql("SELECT gender, AVG(age) AS avg_age FROM occupation_view GROUP BY gender").show()

+------+------------------+
|gender|           avg_age|
+------+------------------+
|     F| 33.81318681318681|
|     M|34.149253731343286|
+------+------------------+



In [9]:
# Pyspark

avg_age_df = occupation.groupBy("gender").agg(avg("age").alias("avg_age"))

# Show the resulting DataFrame
avg_age_df.show()

+------+------------------+
|gender|           avg_age|
+------+------------------+
|     F| 33.81318681318681|
|     M|34.149253731343286|
+------+------------------+



#### Add a new column named "full_name" to the dataset by concatenating the "user_id" and "occupation" columns. Then, rename the "zip_code" column to "postal_code" in the same DataFrame.

In [10]:
# Spark SQL

spark.sql("SELECT *, CONCAT(user_id, occupation) AS full_name FROM occupation_view") \
    .withColumnRenamed("zip_code", "postal_code") \
    .show()

+-------+---+------+-------------+-----------+---------------+
|user_id|age|gender|   occupation|postal_code|      full_name|
+-------+---+------+-------------+-----------+---------------+
|      1| 24|     M|   technician|      85711|    1technician|
|      2| 53|     F|        other|      94043|         2other|
|      3| 23|     M|       writer|      32067|        3writer|
|      4| 24|     M|   technician|      43537|    4technician|
|      5| 33|     F|        other|      15213|         5other|
|      6| 42|     M|    executive|      98101|     6executive|
|      7| 57|     M|administrator|      91344| 7administrator|
|      8| 36|     M|administrator|      05201| 8administrator|
|      9| 29|     M|      student|      01002|       9student|
|     10| 53|     M|       lawyer|      90703|       10lawyer|
|     11| 39|     F|        other|      30329|        11other|
|     12| 28|     F|        other|      06405|        12other|
|     13| 47|     M|     educator|      29206|     13ed

In [11]:
# Pyspark

full_name_df = occupation.withColumn("full_name", concat(col("user_id"), col("occupation"))) \
                         .withColumnRenamed("zip_code", "postal_code")

# Show the resulting DataFrame
full_name_df.show()

+-------+---+------+-------------+-----------+---------------+
|user_id|age|gender|   occupation|postal_code|      full_name|
+-------+---+------+-------------+-----------+---------------+
|      1| 24|     M|   technician|      85711|    1technician|
|      2| 53|     F|        other|      94043|         2other|
|      3| 23|     M|       writer|      32067|        3writer|
|      4| 24|     M|   technician|      43537|    4technician|
|      5| 33|     F|        other|      15213|         5other|
|      6| 42|     M|    executive|      98101|     6executive|
|      7| 57|     M|administrator|      91344| 7administrator|
|      8| 36|     M|administrator|      05201| 8administrator|
|      9| 29|     M|      student|      01002|       9student|
|     10| 53|     M|       lawyer|      90703|       10lawyer|
|     11| 39|     F|        other|      30329|        11other|
|     12| 28|     F|        other|      06405|        12other|
|     13| 47|     M|     educator|      29206|     13ed

#### Filter out rows where occupation is 'technician', select only the "user_id" and "age" columns, and then add a new column "age_diff" that calculates the difference between the user's age and the average age in the dataset.

In [12]:
# Spark SQL

# Assuming you have the "occupation" DataFrame registered as a SQL view named "occupation_view"
average_age = spark.sql("SELECT AVG(age) AS avg_age FROM occupation_view").collect()[0]["avg_age"]

spark.sql("SELECT user_id, age, age - {} AS age_diff FROM occupation_view WHERE occupation = 'technician'"
          .format(average_age)) \
    .show()

+-------+---+------------------+
|user_id|age|          age_diff|
+-------+---+------------------+
|      1| 24|-10.05196182396607|
|      4| 24|-10.05196182396607|
|     44| 26| -8.05196182396607|
|     77| 30| -4.05196182396607|
|    143| 42|  7.94803817603393|
|    197| 55| 20.94803817603393|
|    244| 28| -6.05196182396607|
|    294| 34| -0.05196182396607|
|    311| 32| -2.05196182396607|
|    325| 48| 13.94803817603393|
|    441| 50| 15.94803817603393|
|    456| 24|-10.05196182396607|
|    458| 47| 12.94803817603393|
|    488| 48| 13.94803817603393|
|    545| 27| -7.05196182396607|
|    670| 30| -4.05196182396607|
|    715| 21|-13.05196182396607|
|    717| 24|-10.05196182396607|
|    718| 42|  7.94803817603393|
|    738| 35|  0.94803817603393|
+-------+---+------------------+
only showing top 20 rows



In [13]:
# Pyspark

average_age = occupation.select(avg("age")).collect()[0][0]

age_diff_df = occupation.filter(col("occupation") == "technician") \
                       .select("user_id", "age") \
                       .withColumn("age_diff", col("age") - average_age)

# Show the resulting DataFrame
age_diff_df.show()

+-------+---+-------------------+
|user_id|age|           age_diff|
+-------+---+-------------------+
|      1| 24|-10.051961823966067|
|      4| 24|-10.051961823966067|
|     44| 26| -8.051961823966067|
|     77| 30| -4.051961823966067|
|    143| 42|  7.948038176033933|
|    197| 55| 20.948038176033933|
|    244| 28| -6.051961823966067|
|    294| 34|-0.0519618239660673|
|    311| 32|-2.0519618239660673|
|    325| 48| 13.948038176033933|
|    441| 50| 15.948038176033933|
|    456| 24|-10.051961823966067|
|    458| 47| 12.948038176033933|
|    488| 48| 13.948038176033933|
|    545| 27| -7.051961823966067|
|    670| 30| -4.051961823966067|
|    715| 21|-13.051961823966067|
|    717| 24|-10.051961823966067|
|    718| 42|  7.948038176033933|
|    738| 35| 0.9480381760339327|
+-------+---+-------------------+
only showing top 20 rows



#### Divide the dataset into two DataFrames: one with male users and another with female users. Repartition both DataFrames to have 2 partitions each. Then, union these two DataFrames together and display the resulting DataFrame.

In [18]:
#Using Spark-SQL

male_users = spark.sql("SELECT * FROM occupation_view WHERE gender = 'M'").repartition(2)

male_users.show()

+-------+---+------+----------+--------+
|user_id|age|gender|occupation|zip_code|
+-------+---+------+----------+--------+
|    896| 28|     M|    writer|   91505|
|    156| 25|     M|  educator|   08360|
|    568| 39|     M|  educator|   01915|
|    624| 19|     M|   student|   30067|
|    832| 24|     M|technician|   77042|
|    684| 28|     M|   student|   55414|
|    905| 27|     M|     other|   30350|
|    148| 33|     M|  engineer|   97006|
|    313| 41|     M| marketing|   60035|
|    478| 29|     M|     other|   10019|
|    332| 20|     M|   student|   40504|
|    492| 57|     M|  educator|   94618|
|    833| 34|     M|    writer|   90019|
|    470| 24|     M|programmer|   10021|
|     21| 26|     M|    writer|   30068|
|    265| 26|     M| executive|   84601|
|     33| 23|     M|   student|   27510|
|    133| 53|     M|  engineer|   78602|
|    682| 23|     M|programmer|   55128|
|    650| 42|     M|  engineer|   83814|
+-------+---+------+----------+--------+
only showing top

In [19]:
female_users = spark.sql("SELECT * FROM occupation_view WHERE gender = 'F'").repartition(2)

female_users.show()

+-------+---+------+-------------+--------+
|user_id|age|gender|   occupation|zip_code|
+-------+---+------+-------------+--------+
|    505| 27|     F|        other|   20657|
|    241| 26|     F|      student|   20001|
|    629| 46|     F|        other|   44224|
|    482| 18|     F|      student|   40256|
|    304| 22|     F|      student|   71701|
|    147| 40|     F|    librarian|   02143|
|    354| 29|     F|    librarian|   48197|
|    588| 18|     F|      student|   93063|
|    175| 26|     F|    scientist|   21911|
|    490| 29|     F|       artist|   V5A2B|
|    457| 33|     F|     salesman|   30011|
|    165| 20|     F|        other|   53715|
|    342| 25|     F|        other|   98006|
|    401| 46|     F|   healthcare|   84107|
|    681| 44|     F|    marketing|   97208|
|    238| 42|     F|administrator|   44124|
|     52| 18|     F|      student|   55105|
|    556| 35|     F|     educator|   30606|
|    485| 44|     F|     educator|   95821|
|    126| 28|     F|       lawye

In [20]:
result_df = male_users.union(female_users)
result_df.show()

+-------+---+------+----------+--------+
|user_id|age|gender|occupation|zip_code|
+-------+---+------+----------+--------+
|    896| 28|     M|    writer|   91505|
|    156| 25|     M|  educator|   08360|
|    568| 39|     M|  educator|   01915|
|    624| 19|     M|   student|   30067|
|    832| 24|     M|technician|   77042|
|    684| 28|     M|   student|   55414|
|    905| 27|     M|     other|   30350|
|    148| 33|     M|  engineer|   97006|
|    313| 41|     M| marketing|   60035|
|    478| 29|     M|     other|   10019|
|    332| 20|     M|   student|   40504|
|    492| 57|     M|  educator|   94618|
|    833| 34|     M|    writer|   90019|
|    470| 24|     M|programmer|   10021|
|     21| 26|     M|    writer|   30068|
|    265| 26|     M| executive|   84601|
|     33| 23|     M|   student|   27510|
|    133| 53|     M|  engineer|   78602|
|    682| 23|     M|programmer|   55128|
|    650| 42|     M|  engineer|   83814|
+-------+---+------+----------+--------+
only showing top

In [21]:
# Using pySpark


male_users = occupation.filter(col("gender") == "M").repartition(2)
female_users = occupation.filter(col("gender") == "F").repartition(2)

result_df = female_users.union(male_users)

result_df.show()

+-------+---+------+-------------+--------+
|user_id|age|gender|   occupation|zip_code|
+-------+---+------+-------------+--------+
|    505| 27|     F|        other|   20657|
|    241| 26|     F|      student|   20001|
|    629| 46|     F|        other|   44224|
|    482| 18|     F|      student|   40256|
|    304| 22|     F|      student|   71701|
|    147| 40|     F|    librarian|   02143|
|    354| 29|     F|    librarian|   48197|
|    588| 18|     F|      student|   93063|
|    175| 26|     F|    scientist|   21911|
|    490| 29|     F|       artist|   V5A2B|
|    457| 33|     F|     salesman|   30011|
|    165| 20|     F|        other|   53715|
|    342| 25|     F|        other|   98006|
|    401| 46|     F|   healthcare|   84107|
|    681| 44|     F|    marketing|   97208|
|    238| 42|     F|administrator|   44124|
|     52| 18|     F|      student|   55105|
|    556| 35|     F|     educator|   30606|
|    485| 44|     F|     educator|   95821|
|    126| 28|     F|       lawye

#### Create and fill a new DataFrame named user_ratings with columns user_id and rating max 10 column. Both user_data and user_ratings share the user_id column. Combine these two DataFrames to create a new DataFrame that includes user information and their corresponding ratings. Make sure to keep only the users present in both DataFrames.

In [22]:
from pyspark.sql import SparkSession

# Creating a Spark session
spark = SparkSession.builder.appName("UserRatingsExample").getOrCreate()

#user_data DataFrame with user_id, name, age, and occupation columns
user_data_data = [
    (1, "John"),
    (2, "Alice"),
    (3, "Bob"),
    (4, "Eva"),
    (5, "David"),
    (6, "Sophia"),
    (7, "Michael"),
    (8, "Olivia"),
    (9, "William"),
    (10, "Emma")
]

# Defining the schema for user_data DataFrame
user_data_schema = ["user_id", "name"]

# Create the user_data DataFrame
user_data = spark.createDataFrame(user_data_data, user_data_schema)

# user_ratings DataFrame with user_id and rating columns
user_ratings_data = [
    (1, 9),
    (2, 8),
    (3, 10),
    (4, 7),
    (5, 9),
    (6, 6),
    (7, 10),
    (8, 8),
    (9, 7),
    (10, 9)
]

# Define the schema for user_ratings DataFrame
user_ratings_schema = ["user_id", "rating"]

# Create the user_ratings DataFrame
user_ratings = spark.createDataFrame(user_ratings_data, user_ratings_schema)

# Select relevant columns from "user_ratings"
user_ratings_selected = user_ratings.select("user_id", "rating")

# Join "user_data" and "user_ratings_selected" on "user_id"
combined_df = user_data.join(user_ratings_selected, on="user_id")

# Show the resulting DataFrame
combined_df.show()

23/08/31 08:50:55 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+-------+-------+------+
|user_id|   name|rating|
+-------+-------+------+
|      1|   John|     9|
|      2|  Alice|     8|
|      3|    Bob|    10|
|      4|    Eva|     7|
|      5|  David|     9|
|      6| Sophia|     6|
|      7|Michael|    10|
|      8| Olivia|     8|
|      9|William|     7|
|     10|   Emma|     9|
+-------+-------+------+



In [23]:
test2 = combined_df.select("name")
test2.show(2)

+-----+
| name|
+-----+
| John|
|Alice|
+-----+
only showing top 2 rows



In [26]:
test1 = combined_df.select(expr("name AS destination"))
test1.show(2)

+-----------+
|destination|
+-----------+
|       John|
|      Alice|
+-----------+
only showing top 2 rows



In [27]:
test = combined_df.selectExpr("rating")

test.show()

+------+
|rating|
+------+
|     9|
|     8|
|    10|
|     7|
|     9|
|     6|
|    10|
|     8|
|     7|
|     9|
+------+

